In [1]:
# TODO: go through each activity file and remove wins/losses from Davis Cup (check others like Olympics, Laver Cup, etc.)
# and convert prize money into real prize money
# save to gzipped csv in data/clean/{singles or doubles}/{year}.gz